In [1]:
import pandas as pd
data = pd.read_csv(r"../data/raw/cleaned_speakers_data.csv")
data.head()

,user_id,session_id,sign_in,name,demographic_age,demographic_age_group,demographic_gender,email,location,country,...,pages_visited,conversion_flag,conversion_type,traffic_source,product_purchased,revenue,payment_type,card_type,coupon_applied,bounce_flag
0,U10477,S000001,Email,Victor Navarro-Noël,31,Adult,Female,victornavarronoël251@hotmail.com,Rome,Italy,...,7,0,NCT,Organic,NPP,0.0,NPT,NCAT,ND,1
1,U01536,S000002,Email,王秀云,39,Adult,Female,王秀云617@gmail.com,Madrid,Spain,...,5,0,NCT,Social,NPP,0.0,NPT,NCAT,ND,0
2,U00107,S000003,Guest,Ucchal Sabharwal,68,Old,Male,Not Provided,Manchester,UK,...,7,0,NCT,Organic,NPP,0.0,NPT,NCAT,ND,0
3,U13886,S000004,Email,Virginie Schmitt,72,Old,Female,virginieschmitt827@gmail.com,Sydney,Australia,...,10,0,NCT,Social,NPP,0.0,NPT,NCAT,ND,0
4,U05926,S000005,Email,Cynthia Drake,51,Adult,No Answer,cynthiadrake47@hotmail.com,Mumbai,India,...,6,0,NCT,Organic,NPP,0.0,NPT,NCAT,ND,0


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   user_id                30000 non-null  object 
 1   session_id             30000 non-null  object 
 2   sign_in                30000 non-null  object 
 3   name                   30000 non-null  object 
 4   demographic_age        30000 non-null  int64  
 5   demographic_age_group  30000 non-null  object 
 6   demographic_gender     30000 non-null  object 
 7   email                  30000 non-null  object 
 8   location               30000 non-null  object 
 9   country                30000 non-null  object 
 10  device_type            30000 non-null  object 
 11  timestamp              30000 non-null  object 
 12  variant_group          30000 non-null  object 
 13  time_spent             30000 non-null  float64
 14  pages_visited          30000 non-null  int64  
 15  co

In [3]:
events = pd.read_csv(r'../data/raw/events.csv')

In [4]:
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92048 entries, 0 to 92047
Data columns (total 21 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   event_id                            92048 non-null  object 
 1   user_id                             92048 non-null  object 
 2   session_id                          92048 non-null  object 
 3   event_timestamp                     92048 non-null  object 
 4   event_name                          92048 non-null  object 
 5   user_name                           92048 non-null  object 
 6   user_demographic_age                92048 non-null  int64  
 7   user_demographic_age_group          92048 non-null  object 
 8   user_demographic_gender             92048 non-null  object 
 9   user_email                          92048 non-null  object 
 10  user_location                       92048 non-null  object 
 11  user_country                        92048

In [5]:
data.columns

Index(['user_id', 'session_id', 'sign_in', 'name', 'demographic_age',
       'demographic_age_group', 'demographic_gender', 'email', 'location',
       'country', 'device_type', 'timestamp', 'variant_group', 'time_spent',
       'pages_visited', 'conversion_flag', 'conversion_type', 'traffic_source',
       'product_purchased', 'revenue', 'payment_type', 'card_type',
       'coupon_applied', 'bounce_flag'],
      dtype='object')

In [6]:
SESSIONS_CSV_FILE = r'../data/raw/cleaned_speakers_data.csv' 
df = pd.read_csv(SESSIONS_CSV_FILE)

In [7]:
df

,user_id,session_id,sign_in,name,demographic_age,demographic_age_group,demographic_gender,email,location,country,...,pages_visited,conversion_flag,conversion_type,traffic_source,product_purchased,revenue,payment_type,card_type,coupon_applied,bounce_flag
0,U10477,S000001,Email,Victor Navarro-Noël,31,Adult,Female,victornavarronoël251@hotmail.com,Rome,Italy,...,7,0,NCT,Organic,NPP,0.00,NPT,NCAT,ND,1
1,U01536,S000002,Email,王秀云,39,Adult,Female,王秀云617@gmail.com,Madrid,Spain,...,5,0,NCT,Social,NPP,0.00,NPT,NCAT,ND,0
2,U00107,S000003,Guest,Ucchal Sabharwal,68,Old,Male,Not Provided,Manchester,UK,...,7,0,NCT,Organic,NPP,0.00,NPT,NCAT,ND,0
3,U13886,S000004,Email,Virginie Schmitt,72,Old,Female,virginieschmitt827@gmail.com,Sydney,Australia,...,10,0,NCT,Social,NPP,0.00,NPT,NCAT,ND,0
4,U05926,S000005,Email,Cynthia Drake,51,Adult,No Answer,cynthiadrake47@hotmail.com,Mumbai,India,...,6,0,NCT,Organic,NPP,0.00,NPT,NCAT,ND,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,U08032,S029996,Email,Radhika Gole,15,Teenage,Male,radhikagole401@outlook.com,Sydney,Australia,...,1,1,Signup,Social,NPP,0.00,NPT,NCAT,ND,1
29996,U10440,S029997,Email,Aurélie Jacob,51,Adult,Female,auréliejacob443@protonmail.com,Hong Kong,Hong Kong,...,4,0,NCT,Social,NPP,0.00,NPT,NCAT,ND,0
29997,U07125,S029998,Email,Rodney Miller,15,Teenage,Female,rodneymiller655@gmail.com,Tokyo,Japan,...,2,0,NCT,Social,NPP,0.00,NPT,NCAT,ND,0
29998,U03820,S029999,Email,牛秀珍,73,Old,Female,牛秀珍771@gmail.com,Singapore,Singapore,...,8,1,Purchase,Paid,Sony SRS-XB43,249.99,COD,NCAT,No,0
